# HOMEWORK 07

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.layers import Dense, Conv2D, AveragePooling2D, TimeDistributed, LSTM, GlobalAvgPool2D
from tqdm import tqdm
import datetime

2022-12-19 22:16:57.296541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 22:16:57.605022: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-19 22:16:57.610922: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 22:16:57.610931: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

## Prepare Dataset

In [2]:
# Load MNIST dataset
def load_data():
    (train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)
    return (train_ds, test_ds), ds_info

In [9]:
def new_target_fnc(ds, window_size):
  l = list()
  for i, elem in enumerate(ds):
    if (i % window_size) == 0:
      l.append(int(elem[1]))
    else:
      if (i % 2) == 0:
        l.append(int(l[i-1] + elem[1]))
      else:
        l.append(int(l[i-1] - elem[1]))
  return l

def preprocess(data, batch_size, window_size):
  new_targets = new_target_fnc(data, window_size)
  new_targets = tf.data.Dataset.from_tensor_slices(new_targets)
  data = tf.data.Dataset.zip((data, new_targets))
  data = data.map(lambda img, new_target: (img[0], new_target))
  data = data.map(lambda img, target: (img, tf.one_hot(target, depth=10)))

  data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
  data = data.map(lambda img, target: ((img/128.)-1., target))

  data = data.batch(window_size, drop_remainder=True)
  data = data.batch(batch_size, drop_remainder=True)
  data = data.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
  return data

In [10]:
class CNN(tf.keras.Model):
  def __init__(self, optimizer, loss_function, input_shape):
    super().__init__()
    self.conv1 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'), input_shape=input_shape)
    self.conv2 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'))
    self.pooling1 = TimeDistributed(AveragePooling2D())
    self.conv3 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'))
    self.conv4 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'))
    
    self.globalpooling = TimeDistributed(GlobalAvgPool2D())
    self.out = TimeDistributed(Dense(10, activation='softmax'))

    self.optimizer = optimizer
    self.loss_function = loss_function

    self.metrics_list = [
        tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
        tf.keras.metrics.Mean(name="loss")
    ]

  @tf.function
  def __call__(self, x, training=False):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.pooling1(x)
    x = self.conv3(x)
    x = self.conv4(x)

    x = self.globalpooling(x)
    x = self.out(x)
    return x

  # reset all metrics
  def reset_metrics(self):
      for metric in self.metrics:
          metric.reset_states()

  @tf.function
  def train_step(self, data):
      image, label = data

      with tf.GradientTape() as tape:
          prediction = self(image, training = True)
          loss = self.loss_function(label, prediction)

      gradients = tape.gradient(loss, self.trainable_variables)
      self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))
      self.metrics[0].update_state(label, prediction)
      self.metrics[1].update_state(loss)

  @tf.function
  def test_step(self, data):
      image, label = data
      prediction = self(image, training = False)
      loss = self.loss_function(label, prediction)
      self.metrics[0].update_state(label, prediction)
      self.metrics[1].update_state(loss)


def training_loop(model, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path):
    for epoch in range (epochs):
        model.reset_metrics()

        for data in tqdm(train_ds, position=0, leave=True):
            model.train_step(data)

        with train_summary_writer.as_default():
            tf.summary.scalar(model.metrics[0].name, model.metrics[0].result(), step=epoch)
            tf.summary.scalar(model.metrics[1].name, model.metrics[1].result(), step=epoch)
        
        print("Epoch: ", epoch+1)
        print("Loss: ", model.metrics[1].result().numpy(), "Accuracy: ", model.metrics[0].result().numpy(), "(Train)")
        model.reset_metrics()

        for data in test_ds:
            model.test_step(data)

        with test_summary_writer.as_default():
            tf.summary.scalar(model.metrics[0].name, model.metrics[0].result(), step=epoch)
            tf.summary.scalar(model.metrics[1].name, model.metrics[1].result(), step=epoch)

        print("Loss: ", model.metrics[1].result().numpy(), "Accuracy: ", model.metrics[0].result().numpy(), "(Test)")
    
    model.save_weights(save_path)

In [11]:
batch_size = 32
window_size = 4
(train_ds,test_ds), ds_info = load_data()
train_ds = preprocess(train_ds, batch_size, window_size)
test_ds = preprocess(test_ds, batch_size, window_size)

# for data in train_ds.take(1):
    # print(data[0].shape, data[1])

optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.CategoricalCrossentropy()
cnn = CNN(optimizer=optimizer, loss_function=loss_function, input_shape=(window_size, 28, 28, 1))
epochs = 10

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_path = f"models/{current_time}"
train_log_path = f"logs/{current_time}/train"
test_log_path = f"logs/{current_time}/test"
train_summary_writer = tf.summary.create_file_writer(train_log_path)
test_summary_writer = tf.summary.create_file_writer(test_log_path)
training_loop(cnn, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path)

100%|██████████| 468/468 [00:06<00:00, 73.51it/s]


Epoch:  1
Loss:  1.4735317 Accuracy:  0.23754674 (Train)
Loss:  1.3944105 Accuracy:  0.27323717 (Test)


100%|██████████| 468/468 [00:05<00:00, 92.23it/s] 


Epoch:  2
Loss:  1.3679693 Accuracy:  0.28432158 (Train)
Loss:  1.3539364 Accuracy:  0.29296875 (Test)


100%|██████████| 468/468 [00:05<00:00, 91.56it/s] 


...

100%|██████████| 468/468 [00:05<00:00, 87.53it/s]


Epoch:  6
Loss:  1.3144598 Accuracy:  0.3057058 (Train)
Loss:  1.3232454 Accuracy:  0.30929488 (Test)


100%|██████████| 468/468 [00:05<00:00, 87.00it/s]


Epoch:  7
Loss:  1.3096493 Accuracy:  0.30805957 (Train)
Loss:  1.3203253 Accuracy:  0.3116987 (Test)


 45%|████▌     | 212/468 [00:02<00:02, 89.35it/s]